In [1]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.8 MB/s eta 0:00:00


In [2]:
import keras_tuner as kt
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
import os
import os.path as path
from tqdm import tqdm
from PIL import Image
import scipy as sp
import sklearn
from sklearn.metrics import confusion_matrix
import numpy as np
from PIL import Image, ImageFile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
ImageFile.LOAD_TRUNCATED_IMAGES = True
from keras_tuner import RandomSearch

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Let's build a CNN with a NAS function!

In [4]:
"""
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(Conv2D(filters = 32, kernel_size= (3, 3), activation = 'relu', input_shape = (128, 128, 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(64, (3, 3), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model
"""

"\ndef build_model(hp):\n    model = tf.keras.Sequential()\n    model.add(Conv2D(filters = 32, kernel_size= (3, 3), activation = 'relu', input_shape = (128, 128, 1)))\n    model.add(BatchNormalization())\n    model.add(MaxPooling2D(pool_size = (2, 2)))\n    model.add(Conv2D(64, (3, 3), activation = 'relu'))\n    model.add(BatchNormalization())\n    model.add(MaxPooling2D(pool_size = (2, 2)))\n    model.add(Flatten())\n    model.add(Dense(64, activation = 'relu'))\n    model.add(Dropout(0.5))\n    model.add(Dense(1, activation = 'sigmoid'))\n    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])\n    return model\n"

In [5]:
def build_model(hp):
    model = models.Sequential()

    # CNN Layers
    for i in range(hp.Int('conv_blocks', 1, 3)):
        model.add(layers.Conv2D(filters=hp.Int(f'filters_{i}', 32, 128, step=32), kernel_size=(3, 3), activation='relu'))
        model.add(BatchNormalization())
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Flatten()) # converts multidimensional input array into a one-dimensional vector
    # Dense Layer
    # Dense Layer is where every neuron is connected to a previous neuron
    model.add(layers.Dense(64, activation = 'relu'))
    # RNN Layer
    #model.add(layers.Reshape((1, -1)))  # Adjust this reshape based on input size
    #model.add(layers.LSTM(units=hp.Int('lstm_units', 32, 128, step=32), return_sequences=False))
    # Output Layer

    """
    I feel like we need to make a dropout layer, since overfitting is a worry.
    """
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-3, 1e-4])),
                    loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [6]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    directory='my_dir',
    project_name='cnn_tuning')

# /content/drive/MyDrive/CS171_Files/X-Ray-Datasets/random

In [7]:
data_gen = ImageDataGenerator(1.0/255)
batch_size = 32

In [8]:
train_data = data_gen.flow_from_directory(
        # Training Path
        '/content/drive/MyDrive/CS171_Files/X-Ray-Datasets/random/train',
        target_size=(128, 128),
        color_mode='grayscale',
        class_mode='binary',
        batch_size=batch_size
)

Found 1479 images belonging to 2 classes.


In [9]:
validation_data = data_gen.flow_from_directory(
        # Validation Path
        '/content/drive/MyDrive/CS171_Files/X-Ray-Datasets/random/valid',
        target_size=(128, 128),
        color_mode='grayscale',
        class_mode='binary',
        batch_size=batch_size
)

Found 184 images belonging to 2 classes.


In [10]:
test_data = data_gen.flow_from_directory(
        # Validation Path
        '/content/drive/MyDrive/CS171_Files/X-Ray-Datasets/random/test',
        target_size=(128, 128),
        color_mode='grayscale',
        class_mode='binary',
        batch_size=batch_size
)

Found 184 images belonging to 2 classes.


  ########## NAS Search ##########
  NAS Search and Training

In [13]:
tuner.search(train_data, epochs=10, validation_data=validation_data)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]
best_model.save('/content/drive/My Drive/cnn_randomData_base_model.h5')

# Evaluate the best model
loss, accuracy = best_model.evaluate(validation_data)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

KeyboardInterrupt: 

In [15]:
import os
print(os.getcwd())

/content
